In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("xeno-canto_ca-nv_index.csv")

In [3]:
data.head()

,Unnamed: 0,country,duration_seconds,english_cname,file_id,file_name,file_url,genus,latitude,license,location,longitude,recordist,recordist_url,sonogram_url,species,type,remarks,full_name
0,0,United States,3,Abert's Towhee,17804,XC17804.mp3,https://www.xeno-canto.org/17804/download,Melozone,33.3117,http://creativecommons.org/licenses/by-nc-nd/2.5/,"Cibola National Wildlife Refuge, Cibola, Arizo...",-114.68912,Nathan Pieplow,https://www.xeno-canto.org/contributor/EKKJJJRDJY,https://www.xeno-canto.org/sounds/uploaded/EKK...,aberti,'seet' call,XC17804 © Nathan Pieplow // Cibola National Wi...,Abert's Towhee (Melozone aberti)
1,1,United States,4,Abert's Towhee,177367,XC177367.mp3,https://www.xeno-canto.org/177367/download,Melozone,34.2850,http://creativecommons.org/licenses/by-nc-sa/4.0/,"Bill Williams River NWR, Arizona, United States",-114.06900,Lauren Harter,https://www.xeno-canto.org/contributor/YQNGFTBRRT,https://www.xeno-canto.org/sounds/uploaded/YQN...,aberti,call,XC177367 © Lauren Harter // Bill Williams Rive...,Abert's Towhee (Melozone aberti dumeticola)
2,2,United States,4,Abert's Towhee,145505,XC145505.mp3,https://www.xeno-canto.org/145505/download,Melozone,34.2850,http://creativecommons.org/licenses/by-nc-sa/3.0/,"Bill Williams River NWR, Arizona, United States",-114.06900,Lauren Harter,https://www.xeno-canto.org/contributor/YQNGFTBRRT,https://www.xeno-canto.org/sounds/uploaded/YQN...,aberti,Squeal duet,XC145505 © Lauren Harter // Bill Williams Rive...,Abert's Towhee (Melozone aberti dumeticola)
3,3,United States,5,Abert's Towhee,228159,XC228159.mp3,https://www.xeno-canto.org/228159/download,Melozone,33.1188,http://creativecommons.org/licenses/by-nc-nd/4.0/,"Salton Sea, CA, United States",-115.79450,Peter Boesman,https://www.xeno-canto.org/contributor/OOECIWCSWV,https://www.xeno-canto.org/sounds/uploaded/OOE...,aberti,interaction duet,"XC228159 © Peter Boesman // Salton Sea, CA, Un...",Abert's Towhee (Melozone aberti)
4,4,United States,5,Abert's Towhee,51313,XC51313.mp3,https://www.xeno-canto.org/51313/download,Melozone,36.0628,http://creativecommons.org/licenses/by-nc-sa/3.0/,"Sunset Park, Las Vegas, Nevada, United States",-115.11280,Mike Nelson,https://www.xeno-canto.org/contributor/PWDLINYMKL,https://www.xeno-canto.org/sounds/uploaded/PWD...,aberti,call,"XC51313 © Mike Nelson // Sunset Park, Las Vega...",Abert's Towhee (Melozone aberti dumeticola)


In [4]:
list = data["english_cname"].unique()
bird_list = [bird for bird in list]

In [6]:
def dataframe_difference(df1, df2, which=None):
    """Find rows which are different between two DataFrames."""
    comparison_df = df1.merge(df2,
                              indicator=True,
                              how='outer')
    if which is None:
        diff_df = comparison_df[comparison_df['_merge'] != 'both']
    else:
        diff_df = comparison_df[comparison_df['_merge'] == which]
    return diff_df

In [7]:
temp = data.loc[data['english_cname'] == bird_list[0]]
train_df = data.loc[data['english_cname'] == bird_list[0]].sample(frac=.75, random_state=4)
test_df = dataframe_difference(train_df, temp)

In [8]:
for i in range(1, len(bird_list)):
    temp = data.loc[data['english_cname'] == bird_list[i]]
    train_temp = data.loc[data['english_cname'] == bird_list[i]].sample(frac=.75, random_state=4)
    test_temp = dataframe_difference(train_temp, temp)
    train_df = pd.concat([train_df,train_temp], ignore_index = True)
    test_df = pd.concat([test_df,test_temp], ignore_index = True)

In [41]:
assert len(train_df) + len(test_df) == len(data)

In [12]:
train_df.to_csv("train.csv")

In [13]:
test_df.to_csv("test.csv")

In [14]:
import os
import shutil

In [42]:
test_dir = 'Test/'
train_dir = 'Train/'
source_dir = 'xeno-canto-ca-nv/'

In [17]:
# shutil.move(source_dir, dest_dir)

'xeno-canto-ca-nv 2/Test/Elegant Tern/XC484402.mp3'

In [37]:
for index, row in test_df.iterrows():
    name = row["english_cname"]
    audio = row["file_name"]
    
    if not os.path.exists(f"{test_dir}{name}"):
        os.makedirs(f"{test_dir}{name}")
    
    from_dir = f"{source_dir}{audio}"
    dest_dir =  f"{test_dir}{name}/"
    
    shutil.move(from_dir, dest_dir)
    

In [43]:
for index, row in train_df.iterrows():
    name = row["english_cname"]
    audio = row["file_name"]
    
    if not os.path.exists(f"{train_dir}{name}"):
        os.makedirs(f"{train_dir}{name}")
    
    from_dir = f"{source_dir}{audio}"
    dest_dir =  f"{train_dir}{name}/"
    
    shutil.move(from_dir, dest_dir)